In [1]:
# Imports
import torch
import torchaudio
import torchcrepe
from einops import rearrange
from stable_audio_tools import get_pretrained_model
from stable_audio_tools.inference.generation import generate_diffusion_cond

import numpy as np
import pretty_midi
import sounddevice as sd
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio
import soundfile as sf
from torch.nn import MSELoss

# Custom Helpers
from audio_helpers import render_midi, get_github_audio

device = "cuda" if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else "cpu"
print("Using {}".format(device))

Using cached SoundFont: soundfonts/FluidR3Mono_GM.sf3
Using mps


In [2]:
@torch.enable_grad()
def calculate_pitch(audio, sample_rate):
    # Compute pitch
    audio = audio.mean(dim=0, keepdim=True).to(device=device, dtype=torch.float32) # Down from stereo to mono
    hop = int(sample_rate / 200.)  # 5 ms hop
    pitch, periodicity = torchcrepe.predict(audio, sample_rate, hop_length=hop, fmin=50, fmax=550,
                            model='tiny', # or 'full'
                            batch_size=2048, device=device, return_periodicity=True, decoder=torchcrepe.decode.soft_argmax, differentiable=True)
    # Clean up pitch
    win_l = 3
    periodicity = torchcrepe.filter.median(periodicity, win_l)
    periodicity = torchcrepe.threshold.Silence(-60.)(periodicity, audio, sample_rate, hop)
    pitch = torchcrepe.threshold.At(.5)(pitch, periodicity)
    pitch = torchcrepe.filter.mean(pitch, win_l)
    return pitch

In [3]:
# Download model | Stable Audio Open Small
# `https://huggingface.co/stabilityai/stable-audio-open-small`
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-small")
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]

model = model.to(device)

No module named 'flash_attn'
flash_attn not installed, disabling Flash Attention


/Users/marcocassar/Projects/DLAIE/self/temp/sao-guidance/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [4]:
# if you don't have the audio file download it
get_github_audio("https://github.com/pdx-cs-sound/wavs/raw/refs/heads/main/gc.wav")

target_audio, target_sr = torchaudio.load('data/audio/gc.wav')
if sample_rate != target_sr: # Resample to model rate
    resampler = torchaudio.transforms.Resample(sample_rate, target_sr)
    target_audio = resampler(target_audio)

# Reduce to this really specific time that stable audio open small has
time_sec = 11.888616780045352
target_audio = target_audio[:, :int(time_sec * sample_rate)]
target_pitch = calculate_pitch(target_audio, sample_rate)

print(f"Target length is: {target_audio.shape[1] / sample_rate}")

Using cached file: data/audio/gc.wav
Target length is: 11.888616780045352


In [8]:
from functools import partial

def pitch_callback(model, target_pitch, base_step_scale, alpha, in_dict):
    x, denoised, t = in_dict['x'], in_dict['denoised'], in_dict['t'].detach().float()
    
    with torch.enable_grad():
        x.requires_grad, denoised.requires_grad = True, True
        print(f"t = {in_dict['t']:.3f}, denoised..shape, .requires_grad = {denoised.shape},, {denoised.requires_grad}")

        # PnP-Flow schedule: (1 - t)^alpha
        time_weight = (1.0 - t) ** alpha
        step_scale  = base_step_scale * time_weight
        print(f"time_weight = {float(time_weight):.4f}, step_scale = {float(step_scale):.4f}")

        autoencoder = model._modules['pretransform']._modules.get("model")

        audio = autoencoder.decoder(denoised.half())
        audio = rearrange(audio, "b d n -> d (b n)")
        print("Generated Audio shape:", audio.shape, f"Generated Audio length: {(audio.shape[1]/sample_rate):.2f}")

        # Display audio at each step
        diplay_audio = audio.to(torch.float32).div(torch.max(torch.abs(audio))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()
        display(Audio(diplay_audio.numpy(), rate=sample_rate))
        
        # Compute pitch
        audio = audio.mean(dim=0, keepdim=True).to(device=device, dtype=torch.float32) # Down from stereo to mono
        pitch = calculate_pitch(audio, sample_rate)
        print("pitch.requires_grad =",pitch.requires_grad)
        print("pitch.shape =",pitch.shape)

        loss_fn = MSELoss()
        loss = loss_fn(pitch, target_pitch)
        print("loss.requires_grad =",loss.requires_grad)

        grad_x = torch.autograd.grad(loss, denoised, grad_outputs=torch.ones_like(loss), retain_graph=False, allow_unused=True)[0]
        d_denoised = -step_scale * grad_x
        denoised = denoised + d_denoised

    in_dict['denoised'] = denoised
    in_dict['x'] = x
    return

# Params
base_step_scale = .5
alpha = 1

callback_wrapper = partial(pitch_callback, model, target_pitch, base_step_scale, alpha)

In [ ]:
conditioning = [{
    "prompt": "nylon guitar country scale",  # This prompt is quite bad on small, but small does work
    "seconds_total": time_sec
}]

# Generate stereo audio
output = generate_diffusion_cond(
    model,
    # Marco's Notes:
    # 7 steps works good for sao small, higher than that gets scary
    # If using normal sao higher steps is usually pretty good.
    conditioning=conditioning,
    steps=13,
    cfg_scale=1, # Config of 1 often good for small, higher works on normal
    sample_size=sample_size,
    sigma_min=10,
    sigma_max=300,
    # sampler_type="dpmpp-3m-sde",  # Use this for normal open
    sampler_type="pingpong",  # Use this for small
    device=device,
    callback=callback_wrapper
)

# Rearrange audio batch to a single sequence
output = rearrange(output, "b d n -> d (b n)")

In [ ]:
# Peak normalize, convert to int16
cleaned_output = output.to(torch.float32).div(torch.max(torch.abs(output))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()

# Clip length
#clipped_output = cleaned_output[..., :int(sample_rate * total_seconds)]

Audio(cleaned_output.numpy(), rate=sample_rate)

In [ ]:
# Also display original audio
Audio(target_audio.numpy(), rate=sample_rate)